In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import logging
from scipy.stats import skew, kurtosis

In [2]:
def compute_returns_volatility(top_n, risk_free_rate):
    try:
        # Load datasets
        fundamentals = pd.read_csv('fundamentals.csv')
        prices = pd.read_csv('prices-split-adjusted.csv')
        
        
        # Ensure necessary columns exist
        if 'Ticker Symbol' not in fundamentals.columns or 'Net Income' not in fundamentals.columns:
            raise ValueError("Fundamentals file is missing required columns")
        if 'symbol' not in prices.columns or 'close' not in prices.columns or 'date' not in prices.columns:
            raise ValueError("Prices file is missing required columns")
        
        # Step 1: Identify Top N Profit-Generating Companies
        fundamentals['Net Income'] = fundamentals['Net Income'].fillna(0)
        top_companies = fundamentals.groupby('Ticker Symbol')['Net Income'].sum().nlargest(top_n).index.tolist()
        
        
        
        # Step 2: Filter Prices for Top Companies
        filtered_prices = prices[prices['symbol'].isin(top_companies)].copy()
        
        # Step 3: Calculate Daily Returns
        filtered_prices['daily_return'] = filtered_prices.groupby('symbol')['close'].pct_change()
        
        # Step 4: Compute Annualized Volatility
        volatility = filtered_prices.groupby('symbol')['daily_return'].std() * np.sqrt(252)
        
        # Step 5: Compute Mean Daily Returns & Annualized Returns
        mean_daily_returns = filtered_prices.groupby('symbol')['daily_return'].mean()
        annualized_returns = mean_daily_returns * 252
        
        # Step 6: Compute Risk-Adjusted Metrics
        # skewness = filtered_prices.groupby('symbol')['daily_return'].apply(skew)
        # kurt = filtered_prices.groupby('symbol')['daily_return'].apply(kurtosis)

        # Ensure there are no NaN values in daily returns
        filtered_prices['daily_return'] = filtered_prices['daily_return'].dropna()

        # Compute skewness and kurtosis, handling NaN values
        skewness = filtered_prices.groupby('symbol')['daily_return'].apply(lambda x: skew(x.dropna()))
        kurt = filtered_prices.groupby('symbol')['daily_return'].apply(lambda x: kurtosis(x.dropna()))

        
        # Adjusted Sharpe Ratio (ASR)
        asr = ((annualized_returns - risk_free_rate) / volatility) * (1 + ( ((skewness * 6) + (kurt * 24) - 3) / 24 ) )

        
        # Sortino Ratio
        downside_returns = filtered_prices[filtered_prices['daily_return'] < 0].groupby('symbol')['daily_return']
        downside_std = downside_returns.std() * np.sqrt(252)
        sortino_ratio = (annualized_returns - risk_free_rate) / downside_std
        
        # Omega Ratio (Threshold at risk-free rate)
        omega_ratio = filtered_prices.groupby('symbol')['daily_return'].apply(lambda x: (x[x > risk_free_rate].sum()) / abs(x[x < risk_free_rate].sum()))
        
        # Step 7: Display & Visualize Results
        results = pd.DataFrame({'Annualized Return': annualized_returns, 
                                'Annualized Volatility': volatility, 
                                'Adjusted Sharpe Ratio': asr,
                                'Sortino Ratio': sortino_ratio,
                                'Omega Ratio': omega_ratio})
        results = results.sort_values(by='Annualized Return', ascending=False)
        
        
        print(results)
        
       
        
        
        return results
        
    except Exception as e:
       
        return None


In [3]:
compute_returns_volatility(top_n=10, risk_free_rate=0.02)

        Annualized Return  Annualized Volatility  Adjusted Sharpe Ratio  \
symbol                                                                    
AAPL             0.225074               0.262210               3.978823   
C                0.138091               0.340452               1.982470   
JPM              0.137143               0.271874               1.810457   
WFC              0.132832               0.254683               1.956668   
MSFT             0.126626               0.231882               3.616165   
PFE              0.095121               0.189212               1.345875   
CVX              0.080757               0.218199               0.852139   
XOM              0.056244               0.190376               0.673562   
IBM              0.050755               0.191677               0.866257   
WMT              0.048143               0.163669               2.023695   

        Sortino Ratio  Omega Ratio  
symbol                              
AAPL         1.121903    

,Annualized Return,Annualized Volatility,Adjusted Sharpe Ratio,Sortino Ratio,Omega Ratio
symbol,,,,,
AAPL,0.225074,0.262210,3.978823,1.121903,1.482833
C,0.138091,0.340452,1.982470,0.470887,1.133619
JPM,0.137143,0.271874,1.810457,0.610367,1.215893
WFC,0.132832,0.254683,1.956668,0.641357,1.235385
MSFT,0.126626,0.231882,3.616165,0.670697,1.301865
PFE,0.095121,0.189212,1.345875,0.619900,1.360559
CVX,0.080757,0.218199,0.852139,0.405513,1.207656
XOM,0.056244,0.190376,0.673562,0.275434,1.211645
IBM,0.050755,0.191677,0.866257,0.212052,1.208915
